## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Detecting Themes within Customer Reviews using GenAI


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to detect the prevaling theme of each customer reviews. The themes can then be summarized for each location within the web application.
    - Example Themes:
        - "Good value"
        - "Long Wait Time"

- Notebook Logic:
    1. Gather the reviews that currently have not had their theme extracted.

    2. Create our LLM prompt:
        - The prompt will contain the review text.
        - The prompt will contain the list of themes that we want the LLM to determine matches.
        - The prompt will ask the LLM to provide an explaination of its thought process.
        - The prompt will provide example JSON results so the LLM knows the output format to generate the results.

    3. Execute our LLM prompt using BigFrames

    4. Parse the JSON from the LLM and construct an UPDATE SQL statement so we can update the cutomer review table.
        - We check for valid JSON in case the LLM created incomplete or invalid JSON
        
    5. Execute the SQL statement against BigQuery.

## Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
from bigframes.ml.llm import PaLM2TextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "pkklyaq-nordic-hackathon-1"
REGION = "us"
DATASET_ID = "data_beans_curated"
CONNECTION_NAME = "vertex-ai"

# Connection for BigFrames
connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
# bf.reset_session() # if you need to change the region
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
session = bf.get_global_session()

llm_model = PaLM2TextGenerator(session=session, connection_name=connection)

## Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)

In [ ]:
# Runs a query against BigQuery and waits for it to complete
def RunQuery(sql):
  import time

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

## Detect Customer Themes

In [ ]:
# Gather the unprocessed customer reviews
sql = """SELECT customer_review_id,
                review_text
          FROM `pkklyaq-nordic-hackathon-1.data_beans_curated.customer_review`
         WHERE llm_detected_theme IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

# Loop through the results
for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      Classify the below customer review as one or more of the below themes.
      - Return the results the below json format.
      - Include an explaination for selecting each theme.
      - Do not include double quotes in the explaination.
      - Do not include any special characters, double quotes or "```json" in the json output.

      Themes
      - "Bad Service"
      - "Long Wait Time"
      - "Slow Service"
      - "Dirty"
      - "Overpriced"
      - "Overcrowded"
      - "Noisy Location"
      - "Lack of Allergan Information"
      - "Inconsistent Quality"
      - "Lack of Seating"
      - "No Flavor"
      - "Too weak"
      - "Too strong"
      - "Too bitter"
      - "Limited Menu"
      - "Good Service"
      - "Short Wait Time"
      - "Fast Service"
      - "Clean"
      - "Good value"
      - "Cozy Seating Areas"
      - "Quite Location"
      - "Variety of Milk Alternatives"
      - "Consistent Quality"
      - "Lots of places to sit"
      - "Lots of Flavor"
      - "Good Taste"
      - "Good Selection"

      JSON format: [{ "theme" : "value", "explaination" : "llm explaination" }]
      Sample JSON Response: [{ "theme" : "Fast Service", "explaination" : "The customer got their order fast." }]
      Sample JSON Response: [{ "theme" : "Overpriced", "explaination" : "The customer said it was too expensive." }]

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `pkklyaq-nordic-hackathon-1.data_beans_curated.customer_review`
                    SET llm_detected_theme = JSON'{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)